In [1]:
import itertools
import talib
from chanlun.cl_utils import *
from chanlun.cl_interface import Config

"""
根据缠论数据，选择自己所需要的形态方法集合
"""

direction_types = {"long": ["down"], "short": ["up"]}
mmd_types = {
    "long": ["1buy", "2buy", "3buy", "l2buy", "l3buy"],
    "short": ["1sell", "2sell", "3sell", "l2sell", "l3sell"],
}


def get_opt_types(opt_type: list = []):
    if len(opt_type) == 0:
        opt_type = ["long"]
    opt_direction = list(
        itertools.chain.from_iterable([direction_types[x] for x in opt_type])
    )
    opt_mmd = list(itertools.chain.from_iterable([mmd_types[x] for x in opt_type]))
    return opt_direction, opt_mmd

def xingchenglidu(line: LINE) -> list:
    """
    计算线的行程力度（取绝对值）

    行程力度 = dy / dx
        dy = 终点与起点的绝对值差异
        dx = 线段之间的k线数量
    """
    if line.end.val == line.start.val:
        return 0

    dy = abs(line.end.val - line.start.val)
    dx = line.end.k.k_index - line.start.k.k_index
    return [dy, dx, dy/dx]

def interact():
    """执行后进入repl模式"""
    import code
    code.InteractiveConsole(locals=globals()).interact()
    
if __name__ == "__main__":
    from chanlun.exchange.exchange_tdx import ExchangeTDX
    from chanlun.cl_utils import query_cl_chart_config, web_batch_get_cl_datas
    from chanlun.exchange.exchange import *

    market = "a"
    code = "SH.600038"
    code = "SH.601991"
    # code = "SZ.002491"
    # code = "SZ.002063"
    freq = "2d"

    ex = ExchangeTDX()
    cl_config = query_cl_chart_config(market, code)

        # 获取日K线数据
    klines = ex.klines(code, freq)
    cds = web_batch_get_cl_datas(market, code, {freq: klines}, cl_config)

    cd = cds[0]
    
    xds = cd.get_xds()
    
    xd = xds[-1]
    
    bi_start = xd.start_line
    print(bi_start)
    dy_bi_start = abs(bi_start.end.val - bi_start.start.val)
    dx_bi_start = bi_start.end.k.k_index - bi_start.start.k.k_index
    xc_bi_start = dy_bi_start / dx_bi_start

    bi_end = xd.end_line
    print(bi_end)
    dy_bi_end = abs(bi_end.end.val - bi_end.start.val)
    dx_bi_end = bi_end.end.k.k_index - bi_end.start.k.k_index
    xc_bi_end = dy_bi_end / dx_bi_end

    print(dy_bi_start,dy_bi_end,dx_bi_start,dx_bi_end)
    print(xc_bi_end < xc_bi_start)

    
    if (dy_bi_end > dy_bi_start) and (xc_bi_end < xc_bi_start):
        print("段内笔背驰")
    




Chanlun-Pro 缠论数据计算
https://gitee.com/wang-student/chanlun-pro
运行环境: Python: 3.11
授权信息: A_hia_hia_hia-VIP 
 *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  * 
index: 116 type: up FX: (2024-01-24 15:00:00+08:00 - 2024-05-16 15:00:00+08:00) high: 3.26 low: 2.18 done: True
index: 120 type: up FX: (2024-09-12 15:00:00+08:00 - 2024-10-09 15:00:00+08:00) high: 3.27 low: 2.4 done: True
1.0799999999999996 0.8700000000000001 35 6
False


In [4]:
old_group = "测试选股-20241125-2216"
codes = zx.zx_stocks(zx_group)['code']

测试选股-20241125-2144


In [ ]:
str(cd.get_src_klines()[-2])

In [ ]:
str(cd.get_src_klines()[-1])

In [ ]:
zsds = cd.get_zsds()
zsd = zsds[-1]

In [ ]:
len(cd.get_idx()['macd']['dea'])


In [ ]:
macds_xd = cd.get_idx()['macd']['hist'][xd.start_line.start.k.k_index:xd.end_line.end.k.k_index + 1]

In [ ]:
xd.end_line.end.k.date
reference_date = pd.Timestamp('2024-07-01').tz_localize('UTC')
if reference_date <= xd.end_line.end.k.date:
    print(True)

In [ ]:
macd_xd = cd.get_idx()['macd']['dea'][bi_start.start.k.k_index:bi_start.end.k.k_index + 1]

In [ ]:
macds_xd = cd.get_idx()['macd']['hist'][xd.start_line.start.k.k_index:xd.end_line.end.k.k_index + 1]

In [ ]:
macds_xd

In [ ]:
macd_xd[-3] <= min(macd_xd[:])

In [ ]:
bizss = cd.get_bi_zss()

In [ ]:
(bizss[-1].zs_mmds())

In [ ]:
xd  = xds[-2]

In [ ]:
str(xd)

In [ ]:
bis = cd.get_bis()
for bi in bis:
    if bi.index == xd.start_line.index + 3:
        print(bi)
        print(bi.line_mmds())

In [ ]:
bis = cd.get_bis()
bis1 = bis[xd.start_line.index: xd.end_line.index + 1] 
for bi in bis1:
    print(bi)
    print(bi.line_mmds())

In [ ]:
type((bizss[-1].zs_mmds()[-2]))

In [ ]:
print(bizss[-2])

In [ ]:
bis[xd.start_line.index + 3].mmd_exists(["3sell"])


In [ ]:
bis[xd.start_line.index + 3].get_mmds()[0].name

In [ ]:
bi_start.end.k.date

In [ ]:
len(cd.get_klines())

In [ ]:
xingchenglidu(zsd)

In [ ]:
xingchenglidu(bi_start)

In [ ]:
xingchenglidu(bi_end)

In [ ]:
def xingchenglidu(line: LINE) -> list:
    """
    计算线的行程力度（取绝对值）

    行程力度 = dy / dx
        dy = 终点与起点的绝对值差异
        dx = 线段之间的k线数量
    """
    if line.end.val == line.start.val:
        return 0

    dy = abs(line.end.val - line.start.val)
    dx = line.end.k.k_index - line.start.k.k_index
    return [dy, dx, dy/dx]

def compare_xingcheng_ld(one_line: LINE, two_line: LINE):
    """
    比较两个线的力度，后者小于前者，返回 True
    :param one_line:
    :param two_line:
    :return:
    """
    dy1, dx1, ld1 = xingchenglidu(one_line)
    dy2, dx2, ld2 = xingchenglidu(two_line)
    if ( (dy2 > dy1) and  (ld2 < ld1)):
        return True
    else:
        return False

In [ ]:
compare_xingcheng_ld(zsd.start_line,zsd.end_line)

In [ ]:
zsd.start_line.end.k.date

In [ ]:
bi_start.jiaodu()

In [ ]:
xd.index

In [ ]:
xd.jiaodu()

In [ ]:
xd.start_line.high
# xd.start_line.low

In [ ]:
xd.end_line
xd.end_line.high
xd.end_line.low

In [ ]:
fx_start = xd.start
fx_end = xd.end

In [ ]:
fx_start.klines

In [ ]:
k_start = fx_start.k
k_end = fx_end.k

In [ ]:
k_start.k_index

In [ ]:
k_end.k_index

In [ ]:
diff = k_end.k_index - k_start.k_index


In [ ]:
diff